In [14]:
from collections import defaultdict
import subprocess
import os
from itertools import combinations_with_replacement
import pandas as pd
import numpy as np
import re
class collect_data:
    def __init__(self,file):
        self.data=[]
        if type(file)==str:
            with open(file,'r') as inpt:
                for i in inpt:
                    self.data.append(i.rstrip())
        elif type(file)==list:
            self.data=file
        self.data=self.ml_sl()
        
    def ml_sl(self):
        daat,temp=[],''
        for i in self.data:
            if i.startswith('>'):
                daat.append(temp)
                daat.append(i)
                temp=''
            else:
                temp=temp+i.rstrip()
        daat.append(temp)
        return daat[1:]
    
    def classes(self,part):
        dic_data=defaultdict(list)
        for i in range(0,len(self.data),2):
            if self.data[i].startswith('>'):
                j=self.data[i].split('$')
                dic_data[j[part]].extend([self.data[i],self.data[i+1]])
        return dict(dic_data)
    
class HMM:
    def __init__(self,folder,data):
        self.data=data
        for i,j in self.data.items():
            out=open(f'{name}_hmm/{i}.txt','w')
            for k in j:
                out.write(k+'\n')
            out.close()
            self.create_hmm(i)
        
    def create_hmm(self,file):
        subprocess.run(f'mafft {name}_hmm/{file}.txt > {name}_hmm/{file}.aln',
                       shell=True,stdout=subprocess.PIPE,check=True,universal_newlines=True)
        subprocess.run(f'hmmbuild {name}_hmm/{file}.hmm {name}_hmm/{file}.aln',
                       shell=True,stdout=subprocess.PIPE,check=True,universal_newlines=True)
        print(f'HMM has been created for {file}.....')

class HMM_search:
    def __init__(self,judge,accuse):
        self.judge,self.accuse=judge,accuse
        subprocess.run(f'hmmsearch {name}_hmm/{self.judge}.hmm {name}_hmm/{self.accuse}.txt > {name}_hmm/{self.judge}_prof_{self.accuse}.txt',
                       shell=True,stdout=subprocess.PIPE,check=True,universal_newlines=True)
        print(f'{self.accuse} is ran against {self.judge} profile.....')
        self.mx_mi=self.fetch_values()
        
    def fetch_values(self):
        kix=[]
        with open(f'{name}_hmm/{self.judge}_prof_{self.accuse}.txt') as iinpt:
            for nf in iinpt:
                kix.append(nf.strip())
        ki=[]
        for bf in range(len(kix)):
            if kix[bf].startswith('Scores for complete sequences (score includes all domains):'):
                for cf in range(bf,len(kix)):
                    try:
                        if kix[cf][0].isnumeric():
                            ki.append(float(re.split('\s+',kix[cf])[4]))
                    except IndexError:
                        break
                break
        try:
            return [max(ki),min(ki)]
        except ValueError:
            return [0,0]
                   


In [19]:
a=collect_data('3i2i1i8_GH10_GH11.txt').classes(5) #{0:'acc',1:'org',2:'prt'}
name='_'.join(list(a.keys()))        
try:
    os.mkdir(f'{name}_hmm')
except FileExistsError:
    pass
b=HMM('test',a)
hmm_values={}
djakf=list(combinations_with_replacement(list(a.keys()),2))
for yu in djakf:
    if yu[0]!=yu[1]:
        kkey=yu[0]+'$'+yu[1]
        kkey_rev=yu[1]+'$'+yu[0]
        hmm_values[kkey]=HMM_search(yu[0],yu[1]).mx_mi
        hmm_values[kkey_rev]=HMM_search(yu[1],yu[0]).mx_mi
    else:
        kkey=yu[0]+'$'+yu[1]
        hmm_values[kkey]=HMM_search(yu[0],yu[1]).mx_mi
        
l=list(a.keys())
dda=np.zeros([len(l),len(l)]).astype(str)
dff=pd.DataFrame(dda,index=[l],columns=[l])
for m,n in hmm_values.items():
    m=m.split('$')
    dff.loc[m[1],m[0]]=str(n[0])+' - '+str(n[1])
new_col=[]
for o in l:
    new_col.append(f'{o}_hmm')
dff.columns=new_col
dff.to_excel(f'{name}_hmm/{name}.xlsx',index_label=0)

nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
  301 / 392
done.

Constructing a UPGMA tree (efffree=0) ... 
  390 / 392
done.

Progressive alignment 1/2... 
STEP   316 / 391 
Reallocating..done. *alloclen = 5196
STEP   357 / 391 
Reallocating..done. *alloclen = 7358
STEP   391 / 391 
done.

Making a distance matrix from msa.. 
  300 / 392
done.

Constructing a UPGMA tree (efffree=1) ... 
  390 / 392
done.

Progressive alignment 2/2... 
STEP   350 / 391 
Reallocating..done. *alloclen = 4967
STEP   352 / 391 
Reallocating..done. *alloclen = 6086
STEP   386 / 391 
Reallocating..done. *alloclen = 7359
STEP   391 / 391 
done.

disttbfast (aa) Version 7.490
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.

HMM has been created for GH10.....


nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
  301 / 396
done.

Constructing a UPGMA tree (efffree=0) ... 
  390 / 396
done.

Progressive alignment 1/2... 
STEP   365 / 395 
Reallocating..done. *alloclen = 2791
STEP   390 / 395  h
Reallocating..done. *alloclen = 4094
STEP   395 / 395 
done.

Making a distance matrix from msa.. 
  300 / 396
done.

Constructing a UPGMA tree (efffree=1) ... 
  390 / 396
done.

Progressive alignment 2/2... 
STEP   373 / 395 
Reallocating..done. *alloclen = 2836
STEP   385 / 395 
Reallocating..done. *alloclen = 4042
STEP   395 / 395  h
done.

disttbfast (aa) Version 7.490
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft -

HMM has been created for GH11.....
GH10 is ran against GH10 profile.....
GH11 is ran against GH10 profile.....
GH10 is ran against GH11 profile.....
GH11 is ran against GH11 profile.....


In [18]:
dff #before not removing duplicate entry and multi GH families for same accession number

,GH10_hmm,GH11_hmm
GH10,556.5 - 136.4,200.3 - 200.3
GH11,360.4 - 360.4,420.6 - 130.4


In [21]:
dff #before not removing duplicate entry and multi GH families for same accession number

,GH10_hmm,GH11_hmm
GH10,569.6 - 138.6,0 - 0
GH11,0 - 0,438.8 - 134.9
